# Vanilla DDPM Inpainting — Quantitative Evaluation (Light vs Heavy)

This notebook evaluates the vanilla DDPM inpainting pipeline (`vanilla_second_try.py`) on **N images**
from the **MS COCO val2017** dataset, using **MNIST digit silhouettes** as masks — in two modes:

| Mode | Masked region | Approx. coverage |
|------|--------------|-----------------|
| **Light** | Digit silhouette (erased) | ~10% of image |
| **Heavy** | Everything outside the digit (erased) | ~90% of image |

**Pipeline:** For each image we use the COCO caption as the prompt and measure reconstruction quality
(SSIM, PSNR, LPIPS) on the **inpainted region only**, then show metrics for both modes side-by-side.

**Mask convention (`vanilla_second_try.py`):** `1 = keep, 0 = inpaint`.

**Output layout:**
```
eval_results/
  originals/           ← shared originals (written once)
  light/inpainted/     ← light inpainting outputs
  light/masks/         ← light masks (.pt)
  heavy/inpainted/     ← heavy inpainting outputs
  heavy/masks/         ← heavy masks (.pt)
  checkpoint_light.json
  checkpoint_heavy.json
  metric_distributions_comparison.png
  light_top10_best.png / light_top10_worst.png
  heavy_top10_best.png / heavy_top10_worst.png
```

In [ ]:
import sys, os, json
import numpy as np
import torch
import torchvision
from PIL import Image
import matplotlib.pyplot as plt
from skimage.metrics import structural_similarity as ssim_fn
from tqdm.auto import tqdm
import lpips

# ---- Config ----
N_IMAGES       = 500       # Total images to evaluate
IMAGE_SIZE     = (512, 512)
SEED           = 42
STEPS          = 50           # Reduce to 25 for ~2x speedup
GUIDANCE_SCALE = 7.5
DATA_ROOT      = "./data"
RESULTS_DIR    = "./eval_results"

# Modes: "light" = erase digit silhouette (~10% of image)
#        "heavy" = erase everything except digit (~90% of image)
MODES = ["light", "heavy"]

CHECKPOINT_LIGHT = os.path.join(RESULTS_DIR, "checkpoint_light.json")
CHECKPOINT_HEAVY = os.path.join(RESULTS_DIR, "checkpoint_heavy.json")

COCO_ROOT      = os.path.join(DATA_ROOT, "coco")
COCO_IMG_DIR   = os.path.join(COCO_ROOT, "val2017")
COCO_ANN_FILE  = os.path.join(COCO_ROOT, "annotations", "captions_val2017.json")
COCO_INST_FILE = os.path.join(COCO_ROOT, "annotations", "instances_val2017.json")

# Shared originals dir (written once, used by both modes)
os.makedirs(os.path.join(RESULTS_DIR, "originals"), exist_ok=True)
# Per-mode output dirs
for mode in MODES:
    os.makedirs(os.path.join(RESULTS_DIR, mode, "inpainted"), exist_ok=True)
    os.makedirs(os.path.join(RESULTS_DIR, mode, "masks"),     exist_ok=True)

torch.manual_seed(SEED)
np.random.seed(SEED)

print(f"Config: N_IMAGES={N_IMAGES}, SIZE={IMAGE_SIZE}, STEPS={STEPS}, GUIDANCE={GUIDANCE_SCALE}")
print(f"Output dirs: {RESULTS_DIR}/{{light,heavy}}/{{inpainted,masks}}")

In [ ]:
import urllib.request
import zipfile

# ---- reporthook for download progress ----
_pbar = None

def reporthook(block_num, block_size, total_size):
    global _pbar
    if _pbar is None:
        _pbar = tqdm(total=total_size, unit="B", unit_scale=True, unit_divisor=1024)
    downloaded = block_num * block_size
    _pbar.n = min(downloaded, total_size)
    _pbar.refresh()
    if downloaded >= total_size:
        _pbar.close()
        _pbar = None

# ---- Download COCO val2017 images (~1 GB) ----
os.makedirs(COCO_ROOT, exist_ok=True)

IMG_URL = "http://images.cocodataset.org/zips/val2017.zip"
ANN_URL = "http://images.cocodataset.org/annotations/annotations_trainval2017.zip"

if not os.path.isdir(COCO_IMG_DIR):
    img_zip = os.path.join(COCO_ROOT, "val2017.zip")
    print("Downloading COCO val2017 images (~1 GB)...")
    urllib.request.urlretrieve(IMG_URL, img_zip, reporthook)
    print("Extracting images...")
    with zipfile.ZipFile(img_zip) as z:
        z.extractall(COCO_ROOT)
    os.remove(img_zip)
    print("Done.")
else:
    print(f"COCO images already present: {COCO_IMG_DIR}")

# ---- Download COCO annotations (~240 MB) ----
if not os.path.exists(COCO_ANN_FILE):
    ann_zip = os.path.join(COCO_ROOT, "annotations_trainval2017.zip")
    print("Downloading COCO annotations (~240 MB)...")
    urllib.request.urlretrieve(ANN_URL, ann_zip, reporthook)
    print("Extracting annotations...")
    with zipfile.ZipFile(ann_zip) as z:
        z.extractall(COCO_ROOT)
    os.remove(ann_zip)
    print("Done.")
else:
    print(f"COCO annotations already present: {COCO_ANN_FILE}")

# ---- Load captions JSON → build img_id -> [captions] mapping ----
with open(COCO_ANN_FILE, "r") as f:
    coco_data = json.load(f)

coco_id_to_filename = {img["id"]: img["file_name"] for img in coco_data["images"]}

coco_captions = {}
for ann in coco_data["annotations"]:
    img_id = ann["image_id"]
    coco_captions.setdefault(img_id, []).append(ann["caption"])

# ---- Build set of image IDs to exclude (person, animal, sports, food) ----
# These supercategories produce images dominated by foreground subjects rather
# than buildings, rooms, scenery, nature, or street/vehicle scenes.
EXCLUDED_SUPERCATEGORIES = {"person", "animal", "sports", "food"}

with open(COCO_INST_FILE, "r") as f:
    inst_data = json.load(f)

excluded_cat_ids = {
    c["id"] for c in inst_data["categories"]
    if c["supercategory"] in EXCLUDED_SUPERCATEGORIES
}
excluded_image_ids = {
    a["image_id"] for a in inst_data["annotations"]
    if a["category_id"] in excluded_cat_ids
}
print(f"Images excluded (person/animal/sports/food): "
      f"{len(excluded_image_ids)} / {len(coco_id_to_filename)}")

# ---- Keep only scene/room/nature/vehicle images that have a caption and exist on disk ----
valid_ids = [
    img_id for img_id, caps in coco_captions.items()
    if img_id not in excluded_image_ids
    and os.path.exists(os.path.join(COCO_IMG_DIR, coco_id_to_filename[img_id]))
]
print(f"Scene/room/nature/vehicle images remaining: {len(valid_ids)}")

# ---- Sample N_IMAGES image IDs deterministically ----
rng = np.random.RandomState(SEED)
sampled_ids = rng.choice(valid_ids, size=N_IMAGES, replace=False).tolist()

# Build ordered list of (filename, first_caption) for the sampled images
coco_samples = [
    (coco_id_to_filename[img_id], coco_captions[img_id][0])
    for img_id in sampled_ids
]

# ---- Load MNIST (test set) ----
mnist = torchvision.datasets.MNIST(
    root=DATA_ROOT, train=False, download=True
)

# ---- Sample MNIST indices deterministically ----
mnist_indices = rng.choice(len(mnist), size=N_IMAGES, replace=False)

print(f"MNIST dataset size:  {len(mnist)}")
print(f"Sampled {N_IMAGES} scene/room/nature/vehicle image-mask pairs.")

In [ ]:
def prepare_mnist_mask(mnist_image: Image.Image, size=IMAGE_SIZE) -> torch.Tensor:
    """
    Light inpainting mask: digit silhouette is the erased region (~10% of image).
    Convention: 1 = keep, 0 = inpaint.
    Digit pixels (bright >127) -> 0 (inpaint), background -> 1 (keep).

    Returns: torch.Tensor of shape (1, 1, H, W)
    """
    mask_pil = mnist_image.resize(size, Image.NEAREST)
    mask_np = np.array(mask_pil)
    # Digit pixels are bright; invert so digit -> 0 (inpaint)
    mask_binary = (mask_np <= 127).astype(np.float32)
    mask_tensor = torch.from_numpy(mask_binary).unsqueeze(0).unsqueeze(0)
    return mask_tensor


def prepare_mnist_mask_heavy(mnist_image: Image.Image, size=IMAGE_SIZE) -> torch.Tensor:
    """
    Heavy inpainting mask: everything OUTSIDE the digit is erased (~90% of image).
    Convention: 1 = keep, 0 = inpaint.
    Digit pixels (bright >127) -> 1 (keep), background -> 0 (inpaint).

    This is the inverse of prepare_mnist_mask — the digit is the only known region.

    Returns: torch.Tensor of shape (1, 1, H, W)
    """
    mask_pil = mnist_image.resize(size, Image.NEAREST)
    mask_np = np.array(mask_pil)
    # Digit pixels are bright; keep digit -> 1, erase background -> 0
    mask_binary = (mask_np > 127).astype(np.float32)
    mask_tensor = torch.from_numpy(mask_binary).unsqueeze(0).unsqueeze(0)
    return mask_tensor


def prepare_coco_image(path: str, size=IMAGE_SIZE) -> Image.Image:
    """Load a COCO image from disk and resize to the target size."""
    return Image.open(path).convert("RGB").resize(size, Image.LANCZOS)


def apply_mask_for_display(image: Image.Image, mask_tensor: torch.Tensor) -> Image.Image:
    """Black out the inpainted region for visualization."""
    img_np = np.array(image).copy()
    mask_np = mask_tensor.squeeze().numpy()  # (H, W), 1=keep, 0=inpaint
    img_np[mask_np == 0] = 0  # black for inpainted region
    return Image.fromarray(img_np)


# ---- Sanity check: display one example for BOTH modes ----
sample_filename, sample_caption = coco_samples[0]
sample_img     = prepare_coco_image(os.path.join(COCO_IMG_DIR, sample_filename))
sample_digit   = mnist[mnist_indices[0]][0]

sample_mask_light = prepare_mnist_mask(sample_digit)
sample_mask_heavy = prepare_mnist_mask_heavy(sample_digit)

sample_masked_light = apply_mask_for_display(sample_img, sample_mask_light)
sample_masked_heavy = apply_mask_for_display(sample_img, sample_mask_heavy)

fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# --- Light row ---
axes[0, 0].imshow(sample_img);                                   axes[0, 0].set_title("Original (COCO)")
axes[0, 1].imshow(sample_mask_light.squeeze(), cmap="gray");     axes[0, 1].set_title("Light mask (1=keep)")
axes[0, 2].imshow(sample_masked_light);                          axes[0, 2].set_title("Masked — light (digit erased)")
axes[0, 0].set_ylabel("Light\n(erase digit)", fontsize=11, rotation=0, labelpad=60, va="center")

# --- Heavy row ---
axes[1, 0].imshow(sample_img);                                   axes[1, 0].set_title("")
axes[1, 1].imshow(sample_mask_heavy.squeeze(), cmap="gray");     axes[1, 1].set_title("Heavy mask (1=keep)")
axes[1, 2].imshow(sample_masked_heavy);                          axes[1, 2].set_title("Masked — heavy (background erased)")
axes[1, 0].set_ylabel("Heavy\n(erase background)", fontsize=11, rotation=0, labelpad=60, va="center")

for ax in axes.flat:
    ax.axis("off")

plt.suptitle(f'Caption: "{sample_caption}"', fontsize=10, y=1.01)
plt.tight_layout()
plt.show()

light_frac = (sample_mask_light == 0).float().mean().item() * 100
heavy_frac = (sample_mask_heavy == 0).float().mean().item() * 100
print(f"Light inpainted region: {light_frac:.1f}% of image")
print(f"Heavy inpainted region: {heavy_frac:.1f}% of image")

In [ ]:
# ---- Import model and inpainting function from vanilla_second_try.py ----
sys.path.insert(0, os.path.abspath("."))
from vanilla_second_try import load_vanilla_model, ddpm_inpaint

device = "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using device: {device}")

print("Loading model...")
pipe = load_vanilla_model(device)
pipe.set_progress_bar_config(disable=True)  # suppress inner per-step bar
print("Model loaded.")

In [ ]:
# ---- LIGHT Inpainting loop (erase digit silhouette ~10% of image) ----
# Saves to: eval_results/originals/, eval_results/light/inpainted/, eval_results/light/masks/

LIGHT_INPAINTED_DIR = os.path.join(RESULTS_DIR, "light", "inpainted")
LIGHT_MASKS_DIR     = os.path.join(RESULTS_DIR, "light", "masks")
ORIGINALS_DIR       = os.path.join(RESULTS_DIR, "originals")

# Check for existing checkpoint
start_idx = 0
if os.path.exists(CHECKPOINT_LIGHT):
    with open(CHECKPOINT_LIGHT, "r") as f:
        checkpoint = json.load(f)
    start_idx = checkpoint.get("completed", 0)
    print(f"[Light] Resuming from image {start_idx}/{N_IMAGES}")

for i in tqdm(range(start_idx, N_IMAGES), initial=start_idx,
              total=N_IMAGES, desc="Light inpainting"):
    filename, prompt = coco_samples[i]
    img_pil = prepare_coco_image(os.path.join(COCO_IMG_DIR, filename))
    mask_tensor = prepare_mnist_mask(mnist[mnist_indices[i]][0])

    inpainted_pil = ddpm_inpaint(
        pipe=pipe,
        image=img_pil,
        mask=mask_tensor,
        prompt=prompt,
        steps=STEPS,
        guidance_scale=GUIDANCE_SCALE,
        seed=SEED + i,
    )

    # Save originals once (shared between modes)
    orig_path = os.path.join(ORIGINALS_DIR, f"{i:04d}.png")
    if not os.path.exists(orig_path):
        img_pil.save(orig_path)
        with open(os.path.join(ORIGINALS_DIR, f"{i:04d}.json"), "w") as f:
            json.dump({"caption": prompt}, f)

    inpainted_pil.save(os.path.join(LIGHT_INPAINTED_DIR, f"{i:04d}.png"))
    torch.save(mask_tensor, os.path.join(LIGHT_MASKS_DIR, f"{i:04d}.pt"))

    with open(CHECKPOINT_LIGHT, "w") as f:
        json.dump({"completed": i + 1, "last_prompt": prompt}, f)

print(f"[Light] Done. Inpainted {N_IMAGES} images.")

In [ ]:
# ---- HEAVY Inpainting loop (erase background ~90% of image, keep only digit) ----
# Uses the same COCO image, MNIST digit, prompt, and seed as the corresponding light run.
# Saves to: eval_results/heavy/inpainted/, eval_results/heavy/masks/

HEAVY_INPAINTED_DIR = os.path.join(RESULTS_DIR, "heavy", "inpainted")
HEAVY_MASKS_DIR     = os.path.join(RESULTS_DIR, "heavy", "masks")

# Check for existing checkpoint
start_idx_heavy = 0
if os.path.exists(CHECKPOINT_HEAVY):
    with open(CHECKPOINT_HEAVY, "r") as f:
        checkpoint_h = json.load(f)
    start_idx_heavy = checkpoint_h.get("completed", 0)
    print(f"[Heavy] Resuming from image {start_idx_heavy}/{N_IMAGES}")

for i in tqdm(range(start_idx_heavy, N_IMAGES), initial=start_idx_heavy,
              total=N_IMAGES, desc="Heavy inpainting"):
    filename, prompt = coco_samples[i]
    img_pil = prepare_coco_image(os.path.join(COCO_IMG_DIR, filename))
    mask_tensor = prepare_mnist_mask_heavy(mnist[mnist_indices[i]][0])

    inpainted_pil = ddpm_inpaint(
        pipe=pipe,
        image=img_pil,
        mask=mask_tensor,
        prompt=prompt,
        steps=STEPS,
        guidance_scale=GUIDANCE_SCALE,
        seed=SEED + i,
    )

    # Originals already saved by the light loop; skip if present
    orig_path = os.path.join(ORIGINALS_DIR, f"{i:04d}.png")
    if not os.path.exists(orig_path):
        img_pil.save(orig_path)
        with open(os.path.join(ORIGINALS_DIR, f"{i:04d}.json"), "w") as f:
            json.dump({"caption": prompt}, f)

    inpainted_pil.save(os.path.join(HEAVY_INPAINTED_DIR, f"{i:04d}.png"))
    torch.save(mask_tensor, os.path.join(HEAVY_MASKS_DIR, f"{i:04d}.pt"))

    with open(CHECKPOINT_HEAVY, "w") as f:
        json.dump({"completed": i + 1, "last_prompt": prompt}, f)

print(f"[Heavy] Done. Inpainted {N_IMAGES} images.")

In [ ]:
# ---- Metric helpers ----

lpips_model = lpips.LPIPS(net="alex").to(device)
lpips_model.eval()


def compute_masked_metrics(
    original: Image.Image,
    inpainted: Image.Image,
    mask_tensor: torch.Tensor,
) -> dict:
    """
    Compute SSIM, PSNR, LPIPS only on the inpainted region.
    mask_tensor: (1,1,H,W), 1=keep, 0=inpaint.
    """
    orig_np = np.array(original).astype(np.float64)
    inp_np  = np.array(inpainted).astype(np.float64)
    mask_np = mask_tensor.squeeze().numpy()
    inpaint_mask = (mask_np == 0)

    if not inpaint_mask.any():
        return {"ssim": 1.0, "psnr": float("inf"), "lpips": 0.0}

    # ---- Bounding box of inpainted region ----
    rows = np.any(inpaint_mask, axis=1)
    cols = np.any(inpaint_mask, axis=0)
    rmin, rmax = np.where(rows)[0][[0, -1]]
    cmin, cmax = np.where(cols)[0][[0, -1]]

    crop_orig = orig_np[rmin:rmax + 1, cmin:cmax + 1]
    crop_inp  = inp_np[rmin:rmax + 1, cmin:cmax + 1]

    # ---- PSNR: only on inpainted pixels ----
    masked_orig = orig_np[inpaint_mask]
    masked_inp  = inp_np[inpaint_mask]
    mse = np.mean((masked_orig - masked_inp) ** 2)
    psnr_val = 10.0 * np.log10(255.0 ** 2 / mse) if mse > 0 else float("inf")

    # ---- SSIM: on bounding-box crop ----
    min_dim = min(crop_orig.shape[0], crop_orig.shape[1])
    win_size = min(7, min_dim if min_dim % 2 == 1 else min_dim - 1)
    win_size = max(win_size, 3)

    ssim_val = ssim_fn(
        crop_orig, crop_inp,
        data_range=255.0, channel_axis=2, win_size=win_size
    )

    # ---- LPIPS: on bounding-box crop (avoids black-region artifacts) ----
    crop_orig_f = crop_orig.astype(np.float32)
    crop_inp_f  = crop_inp.astype(np.float32)

    orig_t = torch.from_numpy(crop_orig_f).permute(2, 0, 1).unsqueeze(0) / 127.5 - 1.0
    inp_t  = torch.from_numpy(crop_inp_f).permute(2, 0, 1).unsqueeze(0) / 127.5 - 1.0

    with torch.no_grad():
        lpips_val = lpips_model(orig_t.to(device), inp_t.to(device)).item()

    return {"ssim": ssim_val, "psnr": psnr_val, "lpips": lpips_val}


def run_metrics(inpainted_dir: str, masks_dir: str, originals_dir: str) -> list:
    """
    Load inpainted images, masks, and originals from disk and compute per-image metrics.
    Returns a list of result dicts with keys: idx, original, inpainted, mask, caption, ssim, psnr, lpips.
    """
    results = []
    for i in tqdm(range(N_IMAGES), desc=f"Metrics [{os.path.basename(inpainted_dir)}]"):
        orig_path    = os.path.join(originals_dir,  f"{i:04d}.png")
        inp_path     = os.path.join(inpainted_dir,  f"{i:04d}.png")
        mask_path    = os.path.join(masks_dir,       f"{i:04d}.pt")
        caption_path = os.path.join(originals_dir,  f"{i:04d}.json")

        if not all(os.path.exists(p) for p in [orig_path, inp_path, mask_path]):
            print(f"Stopping at image {i} (files not found). Run the inpainting loop first.")
            break

        orig_img = Image.open(orig_path)
        inp_img  = Image.open(inp_path)
        mask_t   = torch.load(mask_path, weights_only=True)

        caption = ""
        if os.path.exists(caption_path):
            with open(caption_path) as f:
                caption = json.load(f).get("caption", "")

        metrics = compute_masked_metrics(orig_img, inp_img, mask_t)
        results.append({
            "idx": i, "original": orig_img, "inpainted": inp_img,
            "mask": mask_t, "caption": caption, **metrics
        })

    return results


# ---- Compute metrics for both modes ----
originals_dir = os.path.join(RESULTS_DIR, "originals")

results_light = run_metrics(
    inpainted_dir=os.path.join(RESULTS_DIR, "light", "inpainted"),
    masks_dir=os.path.join(RESULTS_DIR, "light", "masks"),
    originals_dir=originals_dir,
)

results_heavy = run_metrics(
    inpainted_dir=os.path.join(RESULTS_DIR, "heavy", "inpainted"),
    masks_dir=os.path.join(RESULTS_DIR, "heavy", "masks"),
    originals_dir=originals_dir,
)

print(f"Light metrics computed for {len(results_light)} images.")
print(f"Heavy metrics computed for {len(results_heavy)} images.")

In [ ]:
# ---- Aggregate statistics and side-by-side comparison ----
from scipy.stats import gaussian_kde

def print_stats_table(results, label):
    ssim_v  = np.array([r["ssim"]  for r in results])
    psnr_v  = np.array([r["psnr"]  for r in results])
    lpips_v = np.array([r["lpips"] for r in results])
    print(f"\n{'=' * 55}")
    print(f"  {label} — {len(results)} images (inpainted region only)")
    print(f"{'=' * 55}")
    print(f"{'Metric':<10} {'Mean':>10} {'Median':>10} {'Std':>10}")
    print(f"{'-' * 55}")
    print(f"{'SSIM':<10} {ssim_v.mean():>10.4f} {np.median(ssim_v):>10.4f} {ssim_v.std():>10.4f}")
    print(f"{'PSNR':<10} {psnr_v.mean():>10.2f} {np.median(psnr_v):>10.2f} {psnr_v.std():>10.2f}")
    print(f"{'LPIPS':<10} {lpips_v.mean():>10.4f} {np.median(lpips_v):>10.4f} {lpips_v.std():>10.4f}")
    print(f"{'=' * 55}")
    return ssim_v, psnr_v, lpips_v

ssim_l,  psnr_l,  lpips_l  = print_stats_table(results_light, "LIGHT inpainting (erase digit ~10%)")
ssim_h,  psnr_h,  lpips_h  = print_stats_table(results_heavy, "HEAVY inpainting (erase background ~90%)")

# ---- Overlay KDE distributions: light vs heavy ----
COLORS = {"light": "steelblue", "heavy": "tomato"}

def plot_kde_overlay(ax, vals_a, vals_b, label_a, label_b, color_a, color_b, xlabel, title):
    for vals, label, color in [(vals_a, label_a, color_a), (vals_b, label_b, color_b)]:
        kde = gaussian_kde(vals, bw_method="scott")
        x = np.linspace(
            min(vals_a.min(), vals_b.min()) - 0.05 * max(np.ptp(vals_a), np.ptp(vals_b)),
            max(vals_a.max(), vals_b.max()) + 0.05 * max(np.ptp(vals_a), np.ptp(vals_b)),
            500,
        )
        y = kde(x)
        ax.plot(x, y, color=color, linewidth=2, label=f"{label} (mean={vals.mean():.3f})")
        ax.fill_between(x, y, alpha=0.20, color=color)
        ax.axvline(vals.mean(), color=color, linestyle="--", linewidth=1.2)
    ax.set_title(title)
    ax.set_xlabel(xlabel)
    ax.set_ylabel("Density")
    ax.legend(fontsize=8)

fig, axes = plt.subplots(1, 3, figsize=(18, 4))
plot_kde_overlay(axes[0], ssim_l,  ssim_h,  "Light", "Heavy",
                 COLORS["light"], COLORS["heavy"], "SSIM",  "SSIM Distribution")
plot_kde_overlay(axes[1], psnr_l,  psnr_h,  "Light", "Heavy",
                 COLORS["light"], COLORS["heavy"], "PSNR (dB)", "PSNR Distribution")
plot_kde_overlay(axes[2], lpips_l, lpips_h, "Light", "Heavy",
                 COLORS["light"], COLORS["heavy"], "LPIPS (lower is better)", "LPIPS Distribution")

plt.suptitle("Light vs Heavy Inpainting — Metric Distributions", fontsize=13, y=1.02)
plt.tight_layout()
plt.savefig(os.path.join(RESULTS_DIR, "metric_distributions_comparison.png"), dpi=150, bbox_inches="tight")
plt.show()

In [ ]:
# ---- Visualization helper ----

def show_top10(results, sort_key="ssim", descending=True, title_prefix="", save_path=None):
    """Display and optionally save a 10-row grid: Original | Masked | Inpainted."""
    sorted_r = sorted(results, key=lambda r: r[sort_key], reverse=descending)
    top10 = sorted_r[:10]

    direction = "highest" if descending else "lowest"
    fig, axes = plt.subplots(10, 3, figsize=(15, 50))
    fig.suptitle(f"{title_prefix} (Top 10 — {direction} {sort_key.upper()})", fontsize=16, y=1.0)

    for row, r in enumerate(top10):
        masked_vis = apply_mask_for_display(r["original"], r["mask"])

        axes[row, 0].imshow(r["original"])
        axes[row, 0].set_title("Original" if row == 0 else "")
        axes[row, 0].set_ylabel(
            f"#{r['idx']}\nSSIM={r['ssim']:.3f}\nPSNR={r['psnr']:.1f}\nLPIPS={r['lpips']:.3f}",
            fontsize=9, rotation=0, labelpad=70, va="center",
        )

        axes[row, 1].imshow(masked_vis)
        axes[row, 1].set_title("Masked" if row == 0 else "")
        axes[row, 1].set_xlabel(f'"{r["caption"]}"', fontsize=8, labelpad=6)

        axes[row, 2].imshow(r["inpainted"])
        axes[row, 2].set_title("Inpainted" if row == 0 else "")

        for col in range(3):
            axes[row, col].axis("off")
        axes[row, 1].xaxis.set_visible(True)
        axes[row, 1].tick_params(bottom=False, labelbottom=True)
        axes[row, 1].set_xticks([])

    plt.tight_layout()
    if save_path:
        plt.savefig(save_path, dpi=150, bbox_inches="tight")
    plt.show()


# ---- Top 10 BEST — Light ----
show_top10(
    results_light,
    sort_key="ssim", descending=True,
    title_prefix="LIGHT Inpainting — Best Results",
    save_path=os.path.join(RESULTS_DIR, "light_top10_best.png"),
)

In [ ]:
# ---- Top 10 WORST — Light ----
show_top10(
    results_light,
    sort_key="ssim", descending=False,
    title_prefix="LIGHT Inpainting — Worst Results",
    save_path=os.path.join(RESULTS_DIR, "light_top10_worst.png"),)

In [ ]:
# ---- Top 10 BEST — Heavy ----
show_top10(
    results_heavy,
    sort_key="ssim", descending=True,
    title_prefix="HEAVY Inpainting — Best Results",
    save_path=os.path.join(RESULTS_DIR, "heavy_top10_best.png"),
)

In [ ]:
# ---- Top 10 WORST — Heavy ----
show_top10(
    results_heavy,
    sort_key="ssim", descending=False,
    title_prefix="HEAVY Inpainting — Worst Results",
    save_path=os.path.join(RESULTS_DIR, "heavy_top10_worst.png"),
)